# load dask cluster with data
load a parquet dataset into a dask cluster

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "dask"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'dask'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import os
import json
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import List, Optional

def load_dask(
    context: MLClientCtx,
    src_data: DataItem,
    dask_key: str = "dask_key",
    inc_cols: List[str] = [],
    index_cols: List[str] = [],
    dask_persist: bool = True,
    refresh_data: bool = True,
    scheduler_key: str = "scheduler"
) -> None:
    """Load dataset into an existing dask cluster
    
    dask jobs define the dask client parameters at the job level, this method will raise an error if no client is detected.
    
    :param context:         the function context
    :param src_data:        url of the data file or partitioned dataset as either
                            artifact DataItem, string, or path object (similar to 
                            pandas read_csv)
    :param dask_key:        destination key of data on dask cluster and artifact store
    :param inc_cols:        include only these columns (very fast)
    :param index_cols:      list of index column names (can be a long-running process)
    :param dask_persist:    (True) should the data be persisted (through the `client.persist` op)
    :param refresh_data:    (False) if the dask_key already exists in the dask cluster, this will 
                            raise an Exception.  Set to True to replace the existing cluster data.
    :param scheduler_key:   (scheduler) the dask scheduler configuration, json also logged as an artifact
    """
    if hasattr(context, "dask_client"):
        dask_client = context.dask_client
    else:
        raise Exception("a dask client was not found in the execution context")
    
    df = src_data.as_df(df_module=dd)

    if dask_persist:
        df = dask_client.persist(df)
        if dask_client.datasets and dask_key in dask_client.datasets:
            dask_client.unpublish_dataset(dask_key)
        dask_client.publish_dataset(df, name=dask_key)
    
    if context:
        context.dask_client = dask_client
        
    # share the scheduler, whether data is persisted or not
    dask_client.write_scheduler_file(scheduler_key+".json")
    
    # we don't use log_dataset here until it can take into account
    # dask origin and apply dask describe.
    context.log_artifact(scheduler_key, local_path=scheduler_key+".json")

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dask", handler="load_dask")

# add metadata (for templates and reuse)
fn.spec.description = "load dask cluster with data"
fn.metadata.categories = ["data-movement", "utils"]
fn.metadata.labels = {"author": "yjb"}
fn.spec.remote = True
fn.spec.replicas = 4
fn.spec.max_replicas = 8
fn.spec.service_type = "NodePort"

fn.save()
fn.export("function.yaml")

[mlrun] 2020-05-03 15:20:55,004 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:20:55,036 function spec saved to path: function.yaml


### test

In [7]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [8]:
from mlrun import NewTask, run_local

task_params = {
    "name":        "tasks load dask cluster with data",
    "params" : {
        "persist"      : True,
        "refresh_data" : True,
        "dask_key"     : "dask_key"}}

In [9]:
run = fn.run(NewTask(**task_params), 
               handler=load_dask, 
               inputs={"src_data" : os.path.join(mlconf.artifact_path, 'iris.parquet') })

[mlrun] 2020-05-03 15:21:02,447 starting run tasks load dask cluster with data uid=c8daa8be140347708531e2f8d80522f2  -> http://mlrun-api:8080
[mlrun] 2020-05-03 15:21:02,491 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:21:07,733 trying dask client at: tcp://mlrun-load-dask-ff2375f9-2.default-tenant:8786
[mlrun] 2020-05-03 15:21:07,752 using remote dask scheduler (mlrun-load-dask-ff2375f9-2) at: tcp://mlrun-load-dask-ff2375f9-2.default-tenant:8786


[mlrun] 2020-05-03 15:21:07,890 log artifact scheduler at /User/artifacts/scheduler.json, size: 182, db: Y

[mlrun] 2020-05-03 15:21:07,927 run ended with state 


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
,...d80522f2,0,May 03 15:21:02,completed,tasks load dask cluster with data,v3io_user=adminkind=daskowner=adminhost=jupyter-6c5fccf844-gxlrw,src_data,persist=Truerefresh_data=Truedask_key=dask_key,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run c8daa8be140347708531e2f8d80522f2  , !mlrun logs c8daa8be140347708531e2f8d80522f2 
[mlrun] 2020-05-03 15:21:08,055 run executed, status=completed


In [10]:
fn.status.to_dict()

{'scheduler_address': 'tcp://mlrun-load-dask-ff2375f9-2.default-tenant:8786',
 'cluster_name': 'mlrun-load-dask-ff2375f9-2',
 'node_ports': {'dashboard': 32392, 'scheduler': 31878}}